# object selection form catalog

currently it's just a simple selection S/N of Ha Hb fluxes > 5

In [1]:
from   astropy.table import Table, vstack, join, setdiff
import numpy         as np

import warnings
from   astropy.io.fits.card import VerifyWarning
warnings.simplefilter('ignore', VerifyWarning)



gn   = Table.read('hlsp_clear_hst_wfc3_gdn_multi_v4.1_clear.fits')
gn_m = Table.read('hlsp_clear_hst_wfc3-acs_gdn-3dhst_multi_v4.6_zout.fits')
gn_cat = join(gn,gn_m,keys_left='ID',keys_right='id',join_type='left',metadata_conflicts='silent')

gs   = Table.read('hlsp_clear_hst_wfc3_gds_multi_v4.1_clear.fits')
gs_m = Table.read('hlsp_clear_hst_wfc3-acs_gds-3dhst_multi_v4.6_zout.fits')
gs_cat = join(gs,gs_m,keys_left='ID',keys_right='id',join_type='left',metadata_conflicts='silent')

cat_lis = vstack([gn_cat,gs_cat],metadata_conflicts='silent')
cat_lis.write('full_object_catalog.fits',overwrite=True)

cat_hasline = cat_lis[np.logical_and(cat_lis['Ha_FLUX'] >0, cat_lis['Hb_FLUX']>0)]
sn_ha = cat_hasline['Ha_FLUX']/cat_hasline['Ha_FLUX_ERR']
sn_hb = cat_hasline['Hb_FLUX']/cat_hasline['Hb_FLUX_ERR']
selection = np.logical_and(sn_ha>5,sn_hb>5)
obj_lis = cat_hasline[selection]

obj_lis.write('obj_lis_selected.fits',overwrite=True)

print('total num of objs in the fields',len(cat_lis))
print('num of objs after s/n selection:',len(obj_lis))


total num of objs in the fields 6048
num of objs after s/n selection: 158


# download spectrum from server.

It is recommended that to run the script downloadSpectra in prompt because of potential stability issue

# Extract line maps from data products

In [130]:
from    astropy.table       import Table
import  numpy               as     np
from    astropy.io          import fits
from    photutils.psf       import matching as match
from    astropy.convolution import convolve_fft 
from    tqdm                import tqdm
import  os
import  gc                                         

#this gives the file name + prefix of an obj in the cat file
def file_name(obj,prefix,filetype='fits'):
    field = obj['subfield'].lower()
    id    = str(obj['ID']).zfill(5)
    return f"hlsp_clear_hst_wfc3_{field}-{id}_g102-g141_v4_{prefix}.{filetype}"


def extract_HaHb(hdu):
    """

    pass objs from obj_lis to extract ha hb lines

    return: HDUlist with the following entry:

    0 primary extension, same as original file

    1 line-fit results

    2 segmentation map

    3 clear filter maps

    4,5 Ha line map & line weight

    6,7 Hb line map & line weight

    """
    #set up a crop of 50x50 pix in the center
    center_size = 50; shape = hdu[5].shape[0]
    #start index: si and end index: ei
    si = (shape - center_size) // 2; 
    ei = si + center_size

    new_file = fits.HDUList()
    #save primary extension
    new_file.append(hdu[0])
    #save line-fit info
    new_file.append(hdu[1])
    """
    select segmentation map [4]
    also save 1 DSCI image for comparison [5]
    """
    for i in [4,5]: 
        hdu[i].data = hdu[i].data[si:ei,si:ei]
        new_file.append(hdu[i])

    #loop to select ha hb line maps
    for image in hdu:
        if image.header.get('EXTTYPE') in ['Ha','Hb'] and (image.name == 'LINE' or image.name == 'LINEWHT'):
            image.data = image.data[si:ei,si:ei]
            image.name = f"{image.name}_{image.header['EXTTYPE']}"
            new_file.append(image)
    return new_file


def data_process(obj):
    try:
        path_data_product   = f"data_products/{file_name(obj,'full')}"
        path_data_extracted = f"data_extracted/{file_name(obj,'extracted')}"

        need_update_from_data_products = False
        need_save_file = False

        
        if need_update_from_data_products == True or not os.path.exists(path_data_extracted):
            extracted = extract_HaHb(fits.open(path_data_product))
        else:
            extracted = fits.open(path_data_extracted,mode='update')
            
        #this part still needs psf matching---------
        
        #save file
        if need_save_file == True: 
            if need_update_from_data_products == True:
                extracted.flush()
            else:
                extracted.writeto(path_data_extracted,overwrite=True)
        return f"{obj['subfield']}-{obj['ID']} saved(updated)"
    
    except Exception as e:
            return f"! {obj['subfield']}-{obj['ID']} failed, error:{e}"


from concurrent.futures import ThreadPoolExecutor, as_completed
def main():
        os.makedirs('data_extracted',exist_ok=True)
        obj_lis = Table.read('obj_lis_selected.fits')
        max_threads=7
        results = []
        with ThreadPoolExecutor(max_threads) as executor:
            futures = {executor.submit(data_process,obj):obj for obj in obj_lis}
            for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                results.append(future.result())
        print(results)

if __name__ == '__main__':
    main()


Processing: 100%|██████████| 158/158 [00:00<00:00, 499.96it/s]

['GN3-35568 saved(updated)', 'GN3-34708 saved(updated)', 'GN3-33115 saved(updated)', 'GN3-32719 saved(updated)', 'GN3-34971 saved(updated)', 'GN3-30204 saved(updated)', 'GN3-34529 saved(updated)', 'GN3-33307 saved(updated)', 'GN3-32660 saved(updated)', 'GN3-33135 saved(updated)', 'GN3-34838 saved(updated)', 'GN3-35042 saved(updated)', 'GN4-28379 saved(updated)', 'GN1-37031 saved(updated)', 'GN3-28121 saved(updated)', 'GN3-35039 saved(updated)', 'GN3-32166 saved(updated)', 'GN7-19504 saved(updated)', 'GN7-17927 saved(updated)', 'GN7-12769 saved(updated)', 'GN7-14281 saved(updated)', 'GN3-35822 saved(updated)', 'GN7-11883 saved(updated)', 'GN7-13777 saved(updated)', 'GN7-19235 saved(updated)', 'GN7-15127 saved(updated)', 'GN7-19659 saved(updated)', 'GN7-13909 saved(updated)', 'GN3-34570 saved(updated)', 'GN7-14850 saved(updated)', 'GN7-14716 saved(updated)', 'GN7-15300 saved(updated)', 'GN7-17532 saved(updated)', 'GN7-13686 saved(updated)', 'GN7-19005 saved(updated)', 'GN7-16041 saved(up